In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
import math

In [2]:
dist = pd.read_csv('dist_data.csv', sep=';')

In [3]:
dist_clean = dist[dist['dist'] != "[[], []]"]

In [4]:
x = dist_clean.iloc[:, :-1].values
y = dist_clean.iloc[:, 1].values

In [5]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.30)

In [6]:
def clean_sample(sample):
    sample_list = sample[0].replace(', [', '; [').split(';')
    sample_x = sample_list[0]
    sample_Px = sample_list[1]

    sample_x_list = []
    sample_Px_list = []
    
    for x in sample_x.split(','):
        sample_x_list.append(x.replace(' ', '').replace('[', '').replace(']', ''))
    for Px in sample_Px.split(','):
        sample_Px_list.append(Px.replace(' ', '').replace('[', '').replace(']', ''))

    dict_sample = {}
    for i in range(len(sample_x_list)):
        dict_sample[sample_x_list[i]] = sample_Px_list[i]
    return dict_sample

In [7]:
def all_keys_dataset1_exist_in_dataset2(dataset1, dataset2, threshold):
    count = 0
    total = len(dataset1.keys())
    for key in dataset1.keys():
        if dataset2.get(key) is None:
            pass
        else:
            count += 1

    if (count/total) * 100 >= float(threshold):
        #print(count, (count/total) * 100, float(threshold))
        return True
    else:
        return False

In [85]:
def calculate_distance(distance_func, min_precent_similarity):
    #count = 0
    result = []
    for i, sample_x_test in enumerate(x_test): 
        result_zikma = []
    
        for j, sample_x_train in enumerate(x_train):
            if all_keys_dataset1_exist_in_dataset2(clean_sample(sample_x_test), clean_sample(sample_x_train), min_precent_similarity):
                Px_list = []
                for xd1, Pxd1 in clean_sample(sample_x_test).items():
                    if xd1 == "" or Pxd1 == "":
                        continue
                    Pxd2 = clean_sample(sample_x_train).get(xd1)
                    if Pxd2 is None:
                        Pxd2 = 0.0
                    #print(xd1, Pxd1, Pxd2)
                    Px_list.append([Pxd1, Pxd2])

                
                sum = distance_func(Px_list)
                result_zikma.append([y_train[j], sum])
                #print(sum)
                
            else:
                continue
        
        result.append([sample_x_test ,result_zikma])
        
        # count += 1
        # if count > 1:
        #     break
    
    return result

In [40]:
def js(Px_list):
    for i in Px_list:
        Mx = (float(i[0]) + float(i[1]))/2
        i.append(Mx)

    PM_list = []
    QM_list = []
    for i in Px_list:
        PM_list.append([i[0], i[2]])
        QM_list.append([i[1], i[2]])

    sum1 = kl(PM_list)
    sum2 = kl(QM_list)

    return (sum1 + sum2)/2


In [69]:
def kl(Px_list):
    sum = 0 
    for i in Px_list:
        Pxd1 = i[0]
        Pxd2 = i[1]
        if float(Pxd1) == 0.0:
            sum += 0.0
            continue
        sum += float(Pxd1) * math.log10(float(Pxd1) / float(Pxd2))
        
    return sum

In [23]:
# def Predict_test1(all_distance_label, threshold):
    # #using max
    # y_pred = []
    # for i in range(len(x_test)):
    #     t = pd.DataFrame(final_result[i][1], columns=['Label', 'Kl Result'])
    #     bests = t.value_counts()
    #     if bests.size == 0:
    #         y_pred.append(0)
    #     else:
    #         y_pred.append(bests.head(1).index[0][0])
    # return y_pred

# def Predict_test2(all_distance_label, threshold):
    # #using sort
    # y_pred = []
    # for i in range(len(x_test)):
    #     t = pd.DataFrame(final_result[i][1], columns=['Label', 'Kl Result'])
    #     grouped_df = t.groupby(['Label', 'Kl Result']).size().reset_index(name='Count')
    #     sorted_df = grouped_df.sort_values(by=['Count', 'Kl Result'], ascending=[False, True])
    #     if t.values.size == 0:
    #         y_pred.append(0)
    #     else:
    #         y_pred.append(sorted_df.head(1)['Label'].values[0])
    # return y_pred    

In [24]:
def Predict(all_distance_label, threshold):
    y_pred = []
    for i in range(len(x_test)):
        t = pd.DataFrame(all_distance_label[i][1], columns=['Label', 'Kl Result'])
        tmp = t[t['Kl Result'] < threshold] #0.99 - 1.01
        grouped_df = tmp.groupby(['Label']).size().reset_index(name='Count')
        sorted_df = grouped_df.sort_values(by=['Count'], ascending=False)
        
        if tmp.values.size == 0:
            y_pred.append(0)
        else:
            y_pred.append(sorted_df.head(1)['Label'].values[0])
    return y_pred

In [25]:
def diffrence_pred_test(y_pred, y_test):
    test = []
    count = 0
    for i, value1 in enumerate(y_pred):
        if value1 != y_test[i]:
            test.append([i, value1, y_test[i]])
            count+=1
    
    return pd.DataFrame(test, columns=['index', 'y_pred', 'y_test'])

In [162]:
def other_percent_similarity():
    final_result = []
    for i in range(30, 101): # one or zero
        final_result.append(calculate_distance(js, i))
    return final_result

In [ ]:
result1 = other_percent_similarity()
result1

In [86]:
final_result = calculate_distance(js, 80)

In [156]:
y_pred = Predict(final_result, 0.0999)

In [157]:
diffrence_pred_test(y_pred, y_test)

,index,y_pred,y_test
0,11,27525,27341
1,18,0,27321
2,36,27525,27523
3,44,27525,27341
4,48,27321,27525
...,...,...,...
150,1550,27525,27341
151,1551,0,27321
152,1561,27523,27321
153,1562,0,27525


In [158]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
res = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(res)
res1 = classification_report(y_test, y_pred)
print("Classification Report:",)
print (res1)
res2 = accuracy_score(y_test,y_pred)
print("Accuracy:",res2)

Confusion Matrix:
[[  0   0   0   0   0   0   0   0]
 [  5 151   0   0   0   0   0   0]
 [ 12   0 359   0  20   8   0   0]
 [  0   0   0   0   0  26   0   0]
 [  2   8   8   0 252   8   0   0]
 [ 22   0  20   0   0 572   0   0]
 [ 11   0   0   0   5   0   1   0]
 [  0   0   0   0   0   0   0  92]]
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
       27311       0.95      0.97      0.96       156
       27321       0.93      0.90      0.91       399
       27341       0.00      0.00      0.00        26
       27523       0.91      0.91      0.91       278
       27525       0.93      0.93      0.93       614
       27652       1.00      0.06      0.11        17
       28090       1.00      1.00      1.00        92

    accuracy                           0.90      1582
   macro avg       0.71      0.60      0.60      1582
weighted avg       0.92      0.90      0.91      1582

Accuracy: 0.902022756005056

/home/arminjp/Documents/Project/pandas/.venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/arminjp/Documents/Project/pandas/.venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/arminjp/Documents/Project/pandas/.venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifie

In [155]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
res = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(res)
res1 = classification_report(y_test, y_pred)
print("Classification Report:",)
print (res1)
res2 = accuracy_score(y_test,y_pred)
print("Accuracy:",res2)

Confusion Matrix:
[[  0   0   0   0   0   0   0   0]
 [  5 151   0   0   0   0   0   0]
 [  8   0 363   0  20   8   0   0]
 [  0   0   0   0   0  26   0   0]
 [  2   8   8   0 251   9   0   0]
 [ 20   0  54   0   0 540   0   0]
 [ 11   0   0   0   5   0   1   0]
 [  0   0   0   0   0   0   0  92]]
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
       27311       0.95      0.97      0.96       156
       27321       0.85      0.91      0.88       399
       27341       0.00      0.00      0.00        26
       27523       0.91      0.90      0.91       278
       27525       0.93      0.88      0.90       614
       27652       1.00      0.06      0.11        17
       28090       1.00      1.00      1.00        92

    accuracy                           0.88      1582
   macro avg       0.70      0.59      0.59      1582
weighted avg       0.90      0.88      0.89      1582

Accuracy: 0.883691529709228

/home/arminjp/Documents/Project/pandas/.venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/arminjp/Documents/Project/pandas/.venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/arminjp/Documents/Project/pandas/.venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifie